In [3]:
using Markdown
using InteractiveUtils
using Markdown, InteractiveUtils
using PlutoUI; PlutoUI.TableOfContents()
using CSV, DataFrames, Statistics, StatsBase, RDatasets, Plots
using EvalMetrics
using Flux

In [4]:
import Pkg; Pkg.add("StatsPlots")

    Updating registry at `C:\Users\Selma Shivute\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Selma Shivute\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Selma Shivute\.julia\environments\v1.8\Manifest.toml`


In [5]:
using CSV, DataFrames, Flux, Statistics
using Flux: @epochs, batchseq, crossentropy, onecold, throttle, train!
using Flux.Losses: mae
using StatsPlots

In [6]:
#import necessary package
import Pkg; Pkg.add("Gadfly")

   Resolving package versions...
  No Changes to `C:\Users\Selma Shivute\.julia\environments\v1.8\Project.toml`
  No Changes to `C:\Users\Selma Shivute\.julia\environments\v1.8\Manifest.toml`


In [7]:
#using the module necesary for the assignment
using Dates, DataFrames, TimeSeries, Gadfly, Dates

In [8]:
using Plots

In [9]:
# Load the data into a dataframe called df
df=CSV.read("C:/Users/Selma Shivute/Documents/NUST/Year 2/Semester 1/AI/dataset/shampoo/sales-of-shampoo.csv", DataFrame)

Row,Month,Sales of shampoo over a three year period
,String7,Float64
1,1-Jan,266.0
2,1-Feb,145.9
3,1-Mar,183.1
4,1-Apr,119.3
5,1-May,180.3
6,1-Jun,168.5
7,1-Jul,231.8
8,1-Aug,224.5
9,1-Sep,192.8


In [10]:
#create a copy of dataframe df into processed_df
processed_df = deepcopy(df)

Row,Month,Sales of shampoo over a three year period
,String7,Float64
1,1-Jan,266.0
2,1-Feb,145.9
3,1-Mar,183.1
4,1-Apr,119.3
5,1-May,180.3
6,1-Jun,168.5
7,1-Jul,231.8
8,1-Aug,224.5
9,1-Sep,192.8


preprocess_data (generic function with 1 method)

In [11]:
import DataFrames.rename!

In [22]:
#rename the column with name Sales of shampoo over a three year period to Sales.
rename!(processed_df, :"Sales of shampoo over a three year period" => :Sales)


Row,Month,Sales
,String7,Float64
1,1-Jan,266.0
2,1-Feb,145.9
3,1-Mar,183.1
4,1-Apr,119.3
5,1-May,180.3
6,1-Jun,168.5
7,1-Jul,231.8
8,1-Aug,224.5
9,1-Sep,192.8


In [14]:
processed_df[!, :Month]

36-element Vector{String7}:
 "1-Jan"
 "1-Feb"
 "1-Mar"
 "1-Apr"
 "1-May"
 "1-Jun"
 "1-Jul"
 "1-Aug"
 "1-Sep"
 "1-Oct"
 "1-Nov"
 "1-Dec"
 "2-Jan"
 ⋮
 "3-Jan"
 "3-Feb"
 "3-Mar"
 "3-Apr"
 "3-May"
 "3-Jun"
 "3-Jul"
 "3-Aug"
 "3-Sep"
 "3-Oct"
 "3-Nov"
 "3-Dec"

In [30]:
#=function preprocess_data(processed_df::DataFrame)
 
    processed_df[!, :Month] = processed_df.date = |>(x-> DateTime(x = "-01", "d-mm-yyyyy")) .|>(x-> Dates.format(x, "m")) 
    processed_df[!, :Year] = processed_df.date = |>(x-> DateTime(x = "-01", "d-mm-yyyyy")) .|>(x-> Dates.format(x, "yyyy")) 
    
    select!(processed_df, Not(:date))
    
    processed_df[!, :Sales] = parse.(Float64, df.Sales)

   
   
    return processed_df
end=#


preprocess_data (generic function with 1 method)

In [39]:
# Define a dictionary to map month abbreviations to numbers
month_map = Dict("Jan" => 1, "Feb" => 2, "Mar" => 3, "Apr" => 4, "May" => 5, "Jun" => 6,
                  "Jul" => 7, "Aug" => 8, "Sep" => 9, "Oct" => 10, "Nov" => 11, "Dec" => 12)

# Split the month and year components
year_strings = SubString.(processed_df[!, :Month], 1, 1)
month_strings = SubString.(processed_df[!, :Month], 3, 5)

# Convert the month and year components to integers
years = parse.(Int, year_strings)
months = [month_map[m] for m in month_strings]

# Create an array of Date objects for each month
num_months = length(years)
dates = [Date(years[i], months[i], 1) for i in 1:num_months]

# Convert the dates to DateTime objects
processed_df[!, :Month] = Dates.DateTime.(dates)


36-element Vector{DateTime}:
 0001-01-01T00:00:00
 0001-02-01T00:00:00
 0001-03-01T00:00:00
 0001-04-01T00:00:00
 0001-05-01T00:00:00
 0001-06-01T00:00:00
 0001-07-01T00:00:00
 0001-08-01T00:00:00
 0001-09-01T00:00:00
 0001-10-01T00:00:00
 0001-11-01T00:00:00
 0001-12-01T00:00:00
 0002-01-01T00:00:00
 ⋮
 0003-01-01T00:00:00
 0003-02-01T00:00:00
 0003-03-01T00:00:00
 0003-04-01T00:00:00
 0003-05-01T00:00:00
 0003-06-01T00:00:00
 0003-07-01T00:00:00
 0003-08-01T00:00:00
 0003-09-01T00:00:00
 0003-10-01T00:00:00
 0003-11-01T00:00:00
 0003-12-01T00:00:00

In [41]:
# Normalize the sales data
sales = processed_df[!, :Sales]
μ, σ = mean(sales), std(sales)
sales = (sales .- μ) ./ σ

36-element Vector{Float64}:
 -0.31288362628797467
 -1.119263959274793
 -0.8694941975170108
 -1.2978627674134222
 -0.8882940720579191
 -0.9675221147660329
 -0.542510665323355
 -0.5915246239478661
 -0.8043660607145783
 -1.27369150014654
  0.16047035768846746
 -0.8506943229761025
 -0.7942946993533774
  ⋮
  0.18195592859236262
  0.8580799879743161
  0.022156994994641667
  0.8506943229761023
  0.5955531684923463
  0.8379372652519143
  1.7651739345731436
  0.6378528862093901
  2.4802405912184073
  1.0924069956449238
  1.8041165318364534
  2.244570735366306

In [55]:
# Extract the sales column as an array
sales = convert(Vector{Float64}, processed_df[!, :Sales])

# Create a time series dataset
ts = TimeArray(sales, dates=processed_df[!, :Month])


LoadError: UndefKeywordError: keyword argument timestamp not assigned

In [66]:
#import Pkg; Pkg.add("Tables")
using Dates, TimeSeries

data = [2.3, 4.5, 6.7, 8.9, 10.1, 12.3, 14.5, 16.7, 18.9, 20.1, 22.3, 24.5]
dates = collect(Date(1, 1, 1):Month(1):Date(3, 12, 1))
periods = Month.(dates)

ta = TimeArray(data, dates; timestamp=periods)


LoadError: MethodError: no method matching TimeArray(::Vector{Float64}, ::Vector{Date}; timestamp=[Month(1), Month(2), Month(3), Month(4), Month(5), Month(6), Month(7), Month(8), Month(9), Month(10)  …  Month(3), Month(4), Month(5), Month(6), Month(7), Month(8), Month(9), Month(10), Month(11), Month(12)])
[0mClosest candidates are:
[0m  TimeArray([91m::D[39m, ::AbstractArray{T, N}) where {T, N, D<:TimeType} at C:\Users\Selma Shivute\.julia\packages\TimeSeries\vYT6q\src\timearray.jl:96[91m got unsupported keyword argument "timestamp"[39m
[0m  TimeArray([91m::D[39m, ::AbstractArray{T, N}, [91m::Vector{S}[39m) where {T, N, D<:TimeType, S<:AbstractString} at deprecated.jl:70[91m got unsupported keyword argument "timestamp"[39m
[0m  TimeArray([91m::D[39m, ::AbstractArray{T, N}, [91m::Vector{Symbol}[39m) where {T, N, D<:TimeType} at C:\Users\Selma Shivute\.julia\packages\TimeSeries\vYT6q\src\timearray.jl:96[91m got unsupported keyword argument "timestamp"[39m
[0m  ...

In [67]:
# Split the dataset into training and testing sets
#train_ratio = 0.8
#train_size = Int(floor(length(ts) * train_ratio))

#train_data = ts[1:train_size]
#test_data = ts[train_size+1:end]


train_data = processed_df[1:25, :]
test_data = processed_df[26:end, :]

# Define the window size and batch size
#window_size = 3
#batch_size = 4

Row,Month,Sales
,DateTime,Float64
1,0003-02-01T00:00:00,440.4
2,0003-03-01T00:00:00,315.9
3,0003-04-01T00:00:00,439.3
4,0003-05-01T00:00:00,401.3
5,0003-06-01T00:00:00,437.4
6,0003-07-01T00:00:00,575.5
7,0003-08-01T00:00:00,407.6
8,0003-09-01T00:00:00,682.0
9,0003-10-01T00:00:00,475.3


In [70]:
using Flux

# Define the TPA-LSTM model
struct TPA_LSTM
    lstm1::Chain
    lstm2::Chain
    tpa::Chain
    output::Dense
end

function TPA_LSTM(input_size::Int, hidden_size::Int, tpa_size::Int, output_size::Int)
    # Create the LSTM layers
    lstm1 = Chain(
        LSTM(input_size, hidden_size),
        LSTM(hidden_size, hidden_size)
    )
    lstm2 = Chain(
        LSTM(hidden_size, hidden_size),
        LSTM(hidden_size, hidden_size)
    )

    # Create the TPA layer
    tpa = Chain(
        Dense(hidden_size, tpa_size, relu),
        softmax
    )

    # Create the output layer
    output = Dense(tpa_size, output_size)

    # Return the TPA-LSTM model
    return TPA_LSTM(lstm1, lstm2, tpa, output)
end

# Initialize the TPA-LSTM model
input_size = 1
hidden_size = 64
tpa_size = 32
output_size = 1
model = TPA_LSTM(input_size, hidden_size, tpa_size, output_size)


TPA_LSTM(Chain(Recur(LSTMCell(1 => 64)), Recur(LSTMCell(64 => 64))), Chain(Recur(LSTMCell(64 => 64)), Recur(LSTMCell(64 => 64))), Chain(Dense(64 => 32, relu), softmax), Dense(32 => 1))

In [72]:
# Train the TPA-LSTM model
# Initialize the TPA-LSTM model
input_size = 1
hidden_size = 64
tpa_size = 32
output_size = 1
model = TPA_LSTM(input_size, hidden_size, tpa_size, output_size)

# Define the loss function and optimizer
loss(x, y) = Flux.mse(model(x), y)
opt = Flux.ADAM()

Adam(0.001, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [74]:
train_data = Matrix(train_data)

25×2 Matrix{Any}:
 0001-01-01T00:00:00  266.0
 0001-02-01T00:00:00  145.9
 0001-03-01T00:00:00  183.1
 0001-04-01T00:00:00  119.3
 0001-05-01T00:00:00  180.3
 0001-06-01T00:00:00  168.5
 0001-07-01T00:00:00  231.8
 0001-08-01T00:00:00  224.5
 0001-09-01T00:00:00  192.8
 0001-10-01T00:00:00  122.9
 0001-11-01T00:00:00  336.5
 0001-12-01T00:00:00  185.9
 0002-01-01T00:00:00  194.3
 0002-02-01T00:00:00  149.5
 0002-03-01T00:00:00  210.1
 0002-04-01T00:00:00  273.3
 0002-05-01T00:00:00  191.4
 0002-06-01T00:00:00  287.0
 0002-07-01T00:00:00  226.0
 0002-08-01T00:00:00  303.6
 0002-09-01T00:00:00  289.9
 0002-10-01T00:00:00  421.6
 0002-11-01T00:00:00  264.5
 0002-12-01T00:00:00  342.3
 0003-01-01T00:00:00  339.7

In [77]:
# Train the model
using Flux
epochs = 100
batch_size = 32
data = [(train_data[i:i+batch_size-1], train_data[i+1:i+batch_size]) for i in 1:length(train_data)-batch_size]
for epoch in 1:epochs
    for (x, y) in data
        Flux.train!(loss, Flux.params(model), [(x, y)], opt)
    end
end


LoadError: MethodError: no method matching (::Flux.LSTMCell{Matrix{Float32}, Matrix{Float32}, Vector{Float32}, Tuple{Matrix{Float32}, Matrix{Float32}}})(::Tuple{Matrix{Float32}, Matrix{Float32}}, ::Vector{Any})
[0mClosest candidates are:
[0m  (::Flux.LSTMCell{I, H, V, <:Tuple{AbstractMatrix{T}, AbstractMatrix{T}}})(::Any, [91m::Union{OneHotArrays.OneHotArray, AbstractVecOrMat{<:AbstractFloat}}[39m) where {I, H, V, T} at C:\Users\Selma Shivute\.julia\packages\Flux\Nzh8J\src\layers\recurrent.jl:310